In [1]:
import os

import xarray as xr
from obstore.store import LocalStore
from virtualizarr.manifests.store import ObjectStoreRegistry

from hrrrparser import HRRRParser

In [2]:
cwd = os.getcwd()
cwd

'/Users/seanharkins/projects/hrrrparser/examples'

In [3]:
file_urls = [
    f"file://{cwd}/hrrr.t22z.wrfsfcf16.grib2",
    f"file://{cwd}/hrrr.t23z.wrfsfcf16.grib2",
]
file_urls

['file:///Users/seanharkins/projects/hrrrparser/examples/hrrr.t22z.wrfsfcf16.grib2',
 'file:///Users/seanharkins/projects/hrrrparser/examples/hrrr.t23z.wrfsfcf16.grib2']

In [4]:
object_store = LocalStore()
registry = ObjectStoreRegistry({"file://": object_store})

In [5]:
parser = HRRRParser()
manifest_stores = []
for url in file_urls:
    manifest_store = parser(url=url, registry=registry)
    manifest_stores.append(manifest_store)
manifest_stores

In [6]:
ds = xr.open_dataset(
    manifest_stores[1], engine="zarr", consolidated=False, zarr_format=3
)
ds

<xarray.Dataset> Size: 3GB
Dimensions:                            (time: 1, step: 1, atm: 1, clt: 1,
                                        sfc: 1, hag: 8, isotherm: 2, isobar: 7,
                                        sigma: 1, entire_atm: 1,
                                        bndry_cloud: 1, lcl: 1, mcl: 1, hcl: 1,
                                        cld_ceiling: 1, clb: 1,
                                        zero_deg_isotherm: 1, htfl: 1,
                                        adiabatic_condensation_lifted: 1,
                                        eqm: 1, lfc: 1, y: 1059, x: 1799)
Coordinates: (12/23)
  * time                               (time) datetime64[s] 8B 2023-07-22T19:...
  * step                               (step) timedelta64[s] 8B 16:00:00
  * atm                                (atm) float64 8B 0.0
  * clt                                (clt) float64 8B 0.0
  * sfc                                (sfc) float64 8B 0.0
  * hag                                (hag) float64 64B 0.0 1.0 ... 2e+03 4e+03
    ...                                 ...
  * htfl                               (htfl) float64 8B 0.0
  * adiabatic_condensation_lifted      (adiabatic_condensation_lifted) float64 8B ...
  * eqm                                (eqm) float64 8B 0.0
  * lfc                                (lfc) float64 8B 0.0
    latitude                           (y, x) float64 15MB ...
    longitude                          (y, x) float64 15MB ...
Dimensions without coordinates: y, x
Data variables: (12/69)
    refc_atm                           (time, atm, y, x) float64 15MB ...
    hail_atm                           (time, atm, y, x) float64 15MB ...
    ltng_atm                           (time, atm, y, x) float64 15MB ...
    tcdc_atm                           (time, atm, y, x) float64 15MB ...
    retop_clt                          (time, clt, y, x) float64 15MB ...
    pres_clt                           (time, clt, y, x) float64 15MB ...
    ...                                 ...
    hgt_htfl                           (time, htfl, y, x) float64 15MB ...
    rh_htfl                            (time, htfl, y, x) float64 15MB ...
    pres_htfl                          (time, htfl, y, x) float64 15MB ...
    hgt_adiabatic_condensation_lifted  (time, adiabatic_condensation_lifted, y, x) float64 15MB ...
    hgt_eqm                            (time, eqm, y, x) float64 15MB ...
    hgt_lfc                            (time, lfc, y, x) float64 15MB ...

In [7]:
ds["tmp_isobar"].isel(y=100, x=100).values

array([[  0.        ,   0.        , 266.30953979, 285.00610352,
        296.15686035, 294.31420898, 291.6489563 ]])

In [10]:
ds["tmp_hag"].isel(hag=2).values

array([[[294.63067627, 294.63067627, 294.63067627, ..., 301.63067627,
         301.63067627, 301.63067627],
        [294.63067627, 294.63067627, 294.69317627, ..., 301.69317627,
         301.69317627, 301.69317627],
        [294.63067627, 294.63067627, 294.63067627, ..., 301.69317627,
         301.69317627, 301.69317627],
        ...,
        [288.13067627, 288.19317627, 288.19317627, ..., 288.81817627,
         288.81817627, 288.75567627],
        [288.13067627, 288.19317627, 288.19317627, ..., 288.81817627,
         288.75567627, 288.69317627],
        [288.13067627, 288.19317627, 288.19317627, ..., 288.75567627,
         288.69317627, 288.69317627]]], shape=(1, 1059, 1799))

In [11]:
ds["longitude"].isel(x=0, y=0).values

array(-122.719528)

In [12]:
vdss = []
for manifest_store in manifest_stores:
    vds = xr.open_dataset(
        manifest_store, engine="zarr", consolidated=False, zarr_format=3
    )
    vdss.append(vds)

In [13]:
combined_vds = xr.combine_nested(
    [vdss], concat_dim=["step", "time"], coords="minimal", compat="override"
)
combined_vds

<xarray.Dataset> Size: 6GB
Dimensions:                            (step: 1, time: 2, atm: 1, y: 1059,
                                        x: 1799, clt: 1, sfc: 1, hag: 8,
                                        isotherm: 2, isobar: 7, sigma: 1,
                                        entire_atm: 1, bndry_cloud: 1, lcl: 1,
                                        mcl: 1, hcl: 1, cld_ceiling: 1, clb: 1,
                                        zero_deg_isotherm: 1, htfl: 1,
                                        adiabatic_condensation_lifted: 1,
                                        eqm: 1, lfc: 1)
Coordinates: (12/23)
  * time                               (time) datetime64[s] 16B 2025-07-10T18...
  * step                               (step) timedelta64[s] 8B 16:00:00
  * atm                                (atm) float64 8B 0.0
  * clt                                (clt) float64 8B 0.0
  * sfc                                (sfc) float64 8B 0.0
  * hag                                (hag) float64 64B 0.0 1.0 ... 2e+03 4e+03
    ...                                 ...
  * htfl                               (htfl) float64 8B 0.0
  * adiabatic_condensation_lifted      (adiabatic_condensation_lifted) float64 8B ...
  * eqm                                (eqm) float64 8B 0.0
  * lfc                                (lfc) float64 8B 0.0
    latitude                           (y, x) float64 15MB ...
    longitude                          (y, x) float64 15MB ...
Dimensions without coordinates: y, x
Data variables: (12/69)
    refc_atm                           (step, time, atm, y, x) float64 30MB -...
    hail_atm                           (step, time, atm, y, x) float64 30MB 0...
    ltng_atm                           (step, time, atm, y, x) float64 30MB 0...
    tcdc_atm                           (step, time, atm, y, x) float64 30MB 1...
    retop_clt                          (step, time, clt, y, x) float64 30MB -...
    pres_clt                           (step, time, clt, y, x) float64 30MB 9...
    ...                                 ...
    hgt_htfl                           (step, time, htfl, y, x) float64 30MB ...
    rh_htfl                            (step, time, htfl, y, x) float64 30MB ...
    pres_htfl                          (step, time, htfl, y, x) float64 30MB ...
    hgt_adiabatic_condensation_lifted  (step, time, adiabatic_condensation_lifted, y, x) float64 30MB ...
    hgt_eqm                            (step, time, eqm, y, x) float64 30MB 2...
    hgt_lfc                            (step, time, lfc, y, x) float64 30MB 3...